In [33]:
from rdflib import URIRef, Literal
from rdflib import Graph as RDFGraph
from rdflib.extras.external_graph_libs import rdflib_to_networkx_graph
import pandas as pd
import networkx as nx

In [29]:
graph = RDFGraph()
graph.parse('./data/imdb-1m.ttl', format="turtle")

<Graph identifier=N30d9d62784164959b4db45c0c840dca9 (<class 'rdflib.graph.Graph'>)>

In [46]:
query = """
CONSTRUCT {
    ?actor ?p ?o .
    ?actor ns1:PlayedIn ?movie
}
WHERE {
    ?actor a schema:Person .
    ?movie a schema:Movie .
    ?movie schema:Actor ?actor
}
"""

actors = graph.query(query)

KeyboardInterrupt: 

In [34]:
g = rdflib_to_networkx_graph(graph)

dict_keys([])